In [ ]:
#!pip install scikit-learn==0.20.0 --upgrade

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import xgboost as xgb
import types
from botocore.client import Config
import ibm_boto3
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

In [ ]:
pd.options.display.max_columns = None
%matplotlib inline

In [1]:
train = pd.read_csv('./Assets/Data/dataset_desafio_2.csv')
train.head()


NameError: name 'pd' is not defined

In [ ]:
# substitua o link abaixo pelo link do seu repositório git (se for o caso)
!git clone https://github.com/vnderlev/sklearn_transforms.git

In [ ]:
!cd sklearn_transforms
!ls -ltr

In [ ]:
!zip -r sklearn_transforms.zip sklearn_transforms

In [ ]:
!pip install sklearn_transforms.zip

In [ ]:
display(train.dtypes)

In [ ]:
train.hist(figsize=(10,8));

In [ ]:
fig, (axis1) = plt.subplots(1,1, figsize=(12,4))
sns.barplot(x='FALTAS', y='PERFIL', data=train)

fig, (axis1) = plt.subplots(1,1, figsize=(12,4))
sns.barplot(x='H_AULA_PRES', y='PERFIL', data=train)

fig, (axis1) = plt.subplots(1,1, figsize=(12,4))
sns.barplot(x='TAREFAS_ONLINE', y='PERFIL', data=train)

In [ ]:
train.isnull().sum()

In [ ]:
m_notago = train['NOTA_GO'].median()
train['NOTA_GO'].fillna(m_notago, inplace=True)

m_ingles = train['INGLES'].median()
train['INGLES'].fillna(m_ingles, inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns

In [ ]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = train[features]
y = train[target]

In [ ]:
X_train, Xtest, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Criação da nossa pipeline para armazenamento no Watson Machine Learning:
my_pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
        ('dtc', DecisionTreeClassifier())
    ]
)

In [ ]:
# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
my_pipeline.fit(X_train, y_train)

In [ ]:
my_pipeline.score(X, y)

In [ ]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_credentials = {
  "apikey": "iJMjQVuVNfxgAvwwt4_n9BSogp9e82x5x1MX7YC8hVa7",
  "iam_apikey_description": "Auto-generated for key 37dee743-9d71-48be-9eb0-42421bf7df41",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/cc91a54686e7485fb839103fcec7b781::serviceid:ServiceId-7fbbd737-677c-4635-839b-acfc31660506",
  "instance_id": "11a1a671-ff4e-48e1-9686-420389546eb4",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [ ]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# Extraindo detalhes da sua instância do Watson Machine Learning

instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

In [ ]:
# Listando todos os artefatos atualmente armazenados na sua instância do WML

clientWML.repository.list()

In [ ]:
# Definição de metadados do nosso pacote com as Transforms personalizadas
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_1",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip criado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

print("\n Lista de artefatos de runtime armazenados no WML:")
clientWML.repository.list()

In [ ]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_1",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalhes do runtime armazenado:")
print(json.dumps(runtime_details, indent=4))

In [ ]:
# Listando todos runtimes armazenados no seu WML:
clientWML.runtimes.list()

In [ ]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-1',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

In [ ]:
# Função para armazenar uma definição de Pipeline no WML
stored_model_details = clientWML.repository.store_model(
    model=my_pipeline,  # `my_pipeline` é a variável criada anteriormente e contém nossa Pipeline já treinada :)
    meta_props=model_meta,  # Metadados definidos na célula anterior
    training_data=None  # Não altere esse parâmetro
)

print("\n Lista de artefatos armazenados no WML:")
clientWML.repository.list()

# Detalhes do modelo hospedado no Watson Machine Learning
print("\n Metadados do modelo armazenado:")
print(json.dumps(stored_model_details, indent=4))

In [ ]:
# O deployment do modelo é finalmente realizado por meio do método ``deployments.create()``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # Não altere esse parâmetro
    name="desafio-2-mbtc2020-deployment-4",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # Não altere esse parâmetro
    deployment_type='online',  # Não altere esse parâmetro
    deployment_format='Core ML',  # Não altere esse parâmetro
    meta_props=model_meta  # Não altere esse parâmetro
)

In [ ]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("A URL de chamada da sua API é: {}".format(model_endpoint_url))

In [ ]:
# Detalhes do deployment realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # esse é o ID do seu deployment!
)

print("Metadados do deployment realizado: \n")
print(json.dumps(deployment_details, indent=4))

In [ ]:
scoring_payload = {
    'fields': [
        "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
        "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    ],
    'values': [
        [
            513949,"Marli Quésia de Oliveira",1,1,1,1,4.3,4.0,3.1,4.9,0,3,4,3,
        ]
    ]
}

print("\n Payload de dados a ser classificada:")
print(json.dumps(scoring_payload, indent=4))

In [ ]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))